In [2]:
import json
import logging
import sys
import os
import math
import shutil
from pathlib import Path
from itertools import chain
# from megatron.data.dataset_utils import get_indexed_dataset_

# import horovod.torch as hvd
# from dotenv import load_dotenv
import torch
import numpy as np
import datasets
from torch.utils.data import DataLoader, DistributedSampler
from datasets import Dataset, load_dataset
from huggingface_hub import hf_hub_download
from sklearn.metrics import f1_score, accuracy_score

sys.path.append('../')
# from lm_experiments_tools import TrainerArgs
from lm_experiments_tools.trainer import Trainer

from torch.nn.utils.rnn import pad_sequence
# from lm_experiments_tools.lm_datasets import get_lm_datasets
from task_utils.contract_nli import process_file
import transformers  # noqa: E402
from transformers import AutoConfig, AutoTokenizer, HfArgumentParser  # noqa: E402

from lm_experiments_tools.utils import collect_run_configuration, get_cls_by_name, get_optimizer  # noqa: E402
import lm_experiments_tools.optimizers as optimizers  # noqa: E402


In [3]:

# load_dotenv()

# logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
#                     level=logging.INFO)
# logger = logging.getLogger(__name__)

# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# # if CUDA_VISIBLE_DEVICES is not set make all gpus visible
# if os.environ.get('CUDA_VISIBLE_DEVICES', None) is None:
#     os.environ['CUDA_VISIBLE_DEVICES'] = ','.join([str(i) for i in range(torch.cuda.device_count())])

# logger.info(f"CUDA_VISIBLE_DEVICES: {os.environ['CUDA_VISIBLE_DEVICES']}")
# # first call to torch.cuda.device_count() sets visible gpus, following calls will not change the result
# logger.info(f"CUDA DEVICE COUNT: {torch.cuda.device_count()}")

# hvd.init()


# # limit # of CPU threads to be used per pytorch worker, otherwise it might use all cpus and throttle gpus
# # > 2 fails cause of https://github.com/pytorch/pytorch/issues/56615
# # need to upgrade to torch>1.8.1
# torch.set_num_threads(4)
# # all gpus set with CUDA_VISIBLE_DEVICES are visible to process, indexing from 0 to ...
# torch.cuda.set_device(hvd.local_rank())

# parser = HfArgumentParser(TrainerArgs)
# parser.add_argument('--task_name', type=str, help="Task name, wikitext, ...")
# parser.add_argument('--validate_only', action='store_true', default=False,
#                     help='Skip training and run only validation. (default: False)')
# parser.add_argument('--working_dir', type=str, default='.',
#                     help='working dir, should be a dir with t5-experiments repo (default: .)')
# parser.add_argument('--seed', type=int, default=42, help='random seed')
# parser.add_argument('--show_valid_examples', type=int, default=0,
#                     help='how many valid examples to show during training (default: 0)')
# parser.add_argument('--input_seq_len', type=int, default=128, help='input sequnce length (default: 128).')
# parser.add_argument('--target_seq_len', type=int, default=16, help='target sequnce length, should be set to '
#                                                                    'max(len(target))+1 for EOS (default: 16).')
# parser.add_argument('--data_n_workers', type=int, default=2, help='number of dataloader workers (default: 2)')

# parser.add_argument('--input_prefix', type=str, default='', help='add task prefix to an input string (default: "")')
# parser.add_argument('--sliding_window', action='store_true', help='use slinding window attentinon mask, '
#                     'eval on last segment only', default=False)

# # model args
# parser.add_argument('--from_pretrained', type=str, help='model name in HF Model Hub (default: "")')
# parser.add_argument('--model_cfg', type=str, help='path to model configuration file (default: "")')
# parser.add_argument('--model_cls', type=str, default='transformers:BertForPreTraining',
#                     help='model class name to use (default: transformers:BertForPreTraining)')
# parser.add_argument('--memory_cell_cls', type=str, default=None, help='cell class for RMT')
# parser.add_argument('--recurrent_wrapper_cls', type=str, default=None, help='recurrent wrapper class for RMT')
# parser.add_argument('--model_cpt', type=str, default=None, help='pretrained model checkpoint path')
# parser.add_argument('--backbone_cls', type=str, default=None,
#                     help='backbone class name to use for RMT')
# parser.add_argument('--model_type', type=str, default='encoder-decoder',
#                     help='model type, encoder, encoder-decoder, decoder, affects preprocessing '
#                          '(default: encoder-decoder)')


# # Aydar # RMT args 
# parser.add_argument('--input_size', type=int, default=None, help='maximal input size of the backbone model')
# parser.add_argument('--num_mem_tokens', type=int, default=None, help='number of memory tokens.')
# parser.add_argument('--max_n_segments', type=int, default=1, help='maximal segment number')
# # parser.add_argument('--sum_loss', action='store_true', default=False,
# #                     help='with this flag task loss from all segments is summed')
# # parser.add_argument('--bptt_depth', type=int, default=-1, help='max number of previous segments in gradient computation.')
# # parser.add_argument('--segment_ordering', type=str, help='segment order', default='regular',
# #                     choices=['regular', 'reversed', 'bidirectional', 'repeat_first', 'last_memory_only'])
# # parser.add_argument('--memory_forward_func', type=str, help='path to memory forward funсtion script', default=None)
# # parser.add_argument('--memory_layers', type=str, help='memory-augmented layer inds or "all" for all layers', default=None)
# # parser.add_argument('--share_memory_layers', action='store_true', help='share weights of memory layers', default=False)
# # parser.add_argument('--reconstruction_loss_coef', type=float, default=None,
# #                     help='reconstuction loss ratio in total loss')
# # # parser.add_argument('--segment_ordering', type=str,help='????', default='regular',
# # #                     choices=['regular', 'reversed', 'bidirectional', 'repeat_first', 'last_memory_only'])
# # parser.add_argument('--retain_graph', action='store_true', help='Retain computation graph during backward pass', default=False)
# # parser.add_argument('--use_truncated_backward', action='store_true', default=False,
# #                     help='whether to use RMT truncated bptt method in backward')
# # parser.add_argument('--k1', type=int, default=-1, help='(not implemented) If not -1, gradient update is done each k1 segments')
# parser.add_argument('--k2', type=int, default=-1, help='number of last segments used by backward')
# parser.add_argument('--freeze_model_weights', action='store_true', default=False,
#                     help='Stop training all model weights except memory layers')
# parser.add_argument('--backbone_cpt', type=str, default=None, help='backbone model checkpoint path')


# # tokenizer
# # todo: add wordpiece tokenizers support?
# parser.add_argument('--tokenizer', type=str, default=None, help='path or name of pre-trained HF Tokenizer')

# # optimizer args
# parser.add_argument('--optimizer', type=str, default='AdamW', help='optimizer name: AdamW, Adafactor. (default: AdamW)')
# parser.add_argument('--weight_decay', type=float, default=0.0, help='optimizer weight decay (default: 0.0)')
# parser.add_argument('--scale_parameter', action='store_true', default=False,
#                     help='Adafactor scale_parameter (default: False)')
# parser.add_argument('--relative_step', action='store_true', default=False,
#                     help='Adafactor relative_step (default: False)')
# parser.add_argument('--warmup_init', action='store_true', default=False,
#                     help='Adafactor warmup_init (default: False)')

In [4]:

def download_metric():
    scrolls_metric_path = hf_hub_download(repo_id="tau/scrolls", filename="metrics/scrolls.py", repo_type="dataset")
    updated_scrolls_metric_path = (
        os.path.dirname(scrolls_metric_path) + os.path.basename(scrolls_metric_path).replace(".", "_") + ".py"
    )
    shutil.copy(scrolls_metric_path, updated_scrolls_metric_path)
    return updated_scrolls_metric_path


scrolls_metric_path = download_metric()

task_to_metric = {
    'gov_report': ['rouge/rouge1', 'rouge/rouge2', 'rouge/rougeL', 'rouge/rougeLsum', 'rouge/geometric_mean'],
    'summ_screen_fd': ['rouge/rouge1', 'rouge/rouge2', 'rouge/rougeL', 'rouge/rougeLsum', 'rouge/geometric_mean'],
    'qmsum': ['rouge/rouge1', 'rouge/rouge2', 'rouge/rougeL', 'rouge/rougeLsum', 'rouge/geometric_mean'],
    'narrative_qa': ['f1'],
    'qasper': ['f1'],
    'quality': ['exact_match'],
    'contract_nli': ['exact_match']
}

tasks_with_duplicates = {'narrative_qa', 'qasper'}


# https://github.com/tau-nlp/scrolls/blob/5bfb8dbaf3a0128ac8c65922096fd95a645f6ba2/baselines/src/utils/duplicates.py#L1
# some tasks have multiple possible labels for single input, drop_duplicates_in_input will collect such labels
def drop_duplicates_in_input(untokenized_dataset):
    indices_to_keep = []
    id_to_idx = {}
    outputs = []
    for i, (id_, output) in enumerate(zip(untokenized_dataset["id"], untokenized_dataset["output"])):
        if id_ in id_to_idx:
            outputs[id_to_idx[id_]].append(output)
            continue
        indices_to_keep.append(i)
        id_to_idx[id_] = len(outputs)
        outputs.append([output])
    untokenized_dataset = untokenized_dataset.select(indices_to_keep).flatten_indices()
    untokenized_dataset = untokenized_dataset.remove_columns("output")
    untokenized_dataset = untokenized_dataset.add_column("outputs", outputs)
    return untokenized_dataset

In [5]:
class Holder:
    def __init__(self):
        pass
    def __getattr__(self, name):
        try:
            return super().__getattr__(name)
        except AttributeError:
            return None

args = Holder()

In [6]:
# model_cpt = '/cephfs/home/bulatov/bulatov/RMT_light/runs/contract_nli/gpt2/linear_adamw_wd1e-03_124-128-1x128_mem2_bs128_regular_bptt-1_from_cpt_0-1/run_1/'
model_cpt = "/cephfs/home/bulatov/bulatov/RMT_light/runs/contract_nli/gpt2/linear_adamw_wd1e-03_118-128-1x128_mem5_bs128_regular_bptt-1_from_cpt_0-1/run_1"
# model_cpt = '/cephfs/home/bulatov/bulatov/RMT_light/runs/contract_nli/gpt2/lr5e-05_linear_adamw_wd1e-03_236-128-2x128_mem5_bs128_iters1500_regular_bptt-2/run_1'

In [7]:
# import json
# with open(model_cpt + '/config.json', 'r') as f:
#     config = json.load(f)

# config

In [8]:
attrs = ("--model_path", "/cephfs/home/bulatov/bulatov/RMT_light/runs/test/cnli",
# "--from_pretrained", "gpt2",
"--from_pretrained", "EleutherAI/pythia-70m-deduped",
"--task_name", "contract_nli",
"--model_type", "decoder",
"--memory_cell_cls", "modeling_rmt.language_modeling:MemoryCell",
"--recurrent_wrapper_cls", "modeling_rmt.language_modeling:RecurrentWrapper",
# "--model_cls",  "transformers:AutoModelForCausalLM",
"--model_cls",  "base_models.modeling_gpt_neox:GPTNeoXForCausalLM",
"--segment_alignment", "right",
# "--model_cpt", model_cpt,
"--optimizer", "AdamW",
"--weight_decay", "0.001",
"--lr", "1e-03", 
"--lr_scheduler", "constant_with_warmup",
"--clip_grad_value", "5.0")

int_attrs = (
    
# "--input_seq_len", "118",
"--input_seq_len", "177",
"--input_size", "128",
"--target_seq_len", "128",
"--num_mem_tokens", "5",
"--max_n_segments" ,"2", 
"--batch_size", "2", 
"--gradient_accumulation_steps", "1",
"--iters", "100",
"--num_warmup_steps", "100",
"--data_n_workers", "2",
"--log_interval", "10",
"--show_valid_examples", "5",
"--early_stopping_patience", "15",
"--seed", "42",
"--k2", "-1",)

In [9]:
for a, v in zip(int_attrs[::2], int_attrs[1::2]):
    setattr(args, a.split('--')[1], int(v))

for a, v in zip(attrs[::2], attrs[1::2]):
    try:
        setattr(args, a.split('--')[1], float(v))
    except ValueError:
        setattr(args, a.split('--')[1], v)

In [10]:

# args = parser.parse_args()
# set current working dir
# args.working_dir = str(Path(args.working_dir).expanduser().absolute())
# os.chdir(args.working_dir)
# if hvd.rank() == 0:
#     logger.info(f'hvd size: {hvd.size()}')
#     logger.info(f'FP16: {args.fp16}')

# if hvd.rank() == 0 and args.model_path is None:
#     logger.warning('model_path is not set: config, logs and checkpoints will not be saved.')

# # create model path and save configuration
# if hvd.rank() == 0 and args.model_path is not None:
#     model_path = Path(args.model_path)
#     if not model_path.exists():
#         Path(model_path).mkdir(parents=True)
#     args_dict = collect_run_configuration(args)
#     # todo: if model path exists and there is config file, write new config file aside
#     json.dump(args_dict, open(model_path/'config.json', 'w'), indent=4)

if not args.from_pretrained:
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer)
else:
    tokenizer = AutoTokenizer.from_pretrained(args.from_pretrained)


loading file vocab.json from cache at None
loading file merges.txt from cache at None
loading file tokenizer.json from cache at /home/jovyan/.cache/huggingface/hub/models--EleutherAI--pythia-70m-deduped/snapshots/e93a9faa9c77e5d09219f6c868bfc7a1bd65593c/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/jovyan/.cache/huggingface/hub/models--EleutherAI--pythia-70m-deduped/snapshots/e93a9faa9c77e5d09219f6c868bfc7a1bd65593c/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/jovyan/.cache/huggingface/hub/models--EleutherAI--pythia-70m-deduped/snapshots/e93a9faa9c77e5d09219f6c868bfc7a1bd65593c/tokenizer_config.json


In [11]:
if args.model_type == 'decoder':
    from torch.nn.utils.rnn import pad_sequence

    tokenizer.add_special_tokens({'additional_special_tokens': ['[GEN]', '[PAD]']})
    gen_token = tokenizer.encode('[GEN]')[0]
    tokenizer.pad_token_id = tokenizer.encode('[PAD]')[0]
    id_pad_value = tokenizer.pad_token_id

    def collate_fn(batch):
        inputs = [b['input'][:args.input_seq_len * 10] for b in batch]
        labels = [b['output'][:args.input_seq_len * 10] for b in batch]

        collated = {}
        inputs = tokenizer.batch_encode_plus(list(inputs), padding=False)
        labels = tokenizer.batch_encode_plus(list(labels), padding=False)

        full_inputs = [torch.tensor(i[:args.input_seq_len - len(l) - 1] + [gen_token] + l) for i, l in zip(inputs['input_ids'], labels['input_ids'])]
        full_inputs = pad_sequence(full_inputs, padding_value=tokenizer.pad_token_id, batch_first=True)

        # gen_inputs = [torch.tensor(i[:args.input_seq_len - 1] + [gen_token]) for i in inputs['input_ids']]
        
        input_ids_generate = torch.ones_like(full_inputs) * tokenizer.pad_token_id
        gen_inputs = [torch.tensor(i[:args.input_seq_len - len(l) - 1] + [gen_token]) for i, l in zip(inputs['input_ids'], labels['input_ids'])]
        gen_inputs = pad_sequence(gen_inputs, padding_value=tokenizer.pad_token_id, batch_first=True)
        input_ids_generate[:, :gen_inputs.shape[1]] = gen_inputs

        
        labels_mask = torch.zeros_like(full_inputs).bool()
        for i, l in enumerate(labels['input_ids']):
            labels_mask[i, -len(l) -1:] = True

        collated['input_ids'] = collated['labels'] = full_inputs
        collated['input_ids_generate'] = input_ids_generate
        collated['labels_mask'] = labels_mask
        collated['attention_mask'] = (collated['input_ids'] != id_pad_value).bool()

        collated['id'] = [b['id'] for b in batch]
        collated['target_text'] = [b['output'] for b in batch]
        return collated

    # def collate_train(batch):
    #     inputs = [b['input'][:args.input_seq_len * 10] for b in batch]
    #     labels = [b['output'][:args.input_seq_len * 10] for b in batch]

    #     collated = {}
    #     inputs = tokenizer.batch_encode_plus(list(inputs), padding=False)
    #     labels = tokenizer.batch_encode_plus(list(labels), padding=False)

    #     full_inputs = [torch.tensor(i[:args.input_seq_len - len(l) - 1] + [gen_token] + l) for i, l in zip(inputs['input_ids'], labels['input_ids'])]
    #     full_inputs = pad_sequence(full_inputs, padding_value=tokenizer.pad_token_id).T
        
    #     labels_mask = torch.zeros_like(full_inputs).bool()
    #     for i, l in enumerate(labels['input_ids']):
    #         labels_mask[i, -len(l) -1:] = True

    #     collated['input_ids'] = collated['labels'] = full_inputs
    #     collated['labels_mask'] = labels_mask
    #     collated['attention_mask'] = (collated['input_ids'] != id_pad_value).bool()
    #     return collated

    # def collate_valid(batch):
    #     inputs = [b['input'][:args.input_seq_len * 10] for b in batch]

    #     collated = {}
    #     inputs = tokenizer.batch_encode_plus(list(inputs), padding=False)
    #     full_inputs = [torch.tensor(i[:args.input_seq_len - 1] + [gen_token]) for i in inputs['input_ids']]
    #     full_inputs = pad_sequence(full_inputs, padding_value=tokenizer.pad_token_id).T
        
    #     collated['input_ids'] = full_inputs
    #     collated['attention_mask'] = (collated['input_ids'] != id_pad_value).bool()
    #     return collated

Assigning ['[GEN]', '[PAD]'] to the additional_special_tokens key of the tokenizer


In [12]:

# get train dataset
# if hvd.rank() == 0:
#     logger.info(f'preparing dataset for: {args.task_name}')
dataset = datasets.load_dataset('tau/scrolls', args.task_name)
train_dataset = dataset['train']
valid_dataset = dataset['validation']
# shuffle train data each epoch (one loop over train_dataset)
# train_sampler = DistributedSampler(train_dataset, rank=hvd.rank(), num_replicas=hvd.size(), shuffle=True,
#                                     drop_last=False, seed=args.seed)
per_worker_batch_size = int(args.batch_size * args.gradient_accumulation_steps)
global_batch_size = int(args.batch_size)
kwargs = {'pin_memory': True, 'num_workers': args.data_n_workers}
train_dataloader = DataLoader(train_dataset, batch_size=per_worker_batch_size, 
# sampler=train_sampler,
                                collate_fn=collate_fn, **kwargs)
                                # collate_fn=collate_train, **kwargs)
# get validation dataset
valid_dataloader = None
# if hvd.rank() == 0:
#     logger.info(f'preparing validation data from: {args.task_name}')
# valid_sampler = DistributedSampler(valid_dataset, rank=hvd.rank(), num_replicas=hvd.size(), shuffle=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=per_worker_batch_size, 
# sampler=valid_sampler,
                                collate_fn=collate_fn, **kwargs)
                                # collate_fn=collate_valid, **kwargs)

# test_sampler = DistributedSampler(test_dataset, rank=hvd.rank(), num_replicas=hvd.size(), shuffle=False)
# test_dataloader = DataLoader(test_dataset, batch_size=per_worker_batch_size, 
# sampler=test_sampler,
                                # collate_fn=collate_fn, **kwargs)
# if args.valid_interval is None:
#     args.valid_interval = args.log_interval


In [13]:
import math
import torch
from torch.nn import CrossEntropyLoss
from transformers.modeling_outputs import CausalLMOutputWithCrossAttentions

class MemoryCell(torch.nn.Module):
    def __init__(self, base_model, num_mem_tokens):
        super().__init__()
        self.model = base_model
        self.create_memory(num_mem_tokens)

    def create_memory(self, num_mem_tokens):
        self.num_mem_tokens = num_mem_tokens
        embeddings = self.model.get_input_embeddings()
        memory_dim =  getattr(self.model.config, 'n_embd', self.model.config.hidden_size)
        memory_weights = torch.randn((num_mem_tokens, memory_dim)) * embeddings.weight.data.std()
        self.register_parameter('memory', torch.nn.Parameter(memory_weights, requires_grad=True))

        self.read_memory_position = range(num_mem_tokens)
        self.write_memory_position = range(-num_mem_tokens, 0)

    def set_memory(self, input_shape):
        memory = self.memory.repeat(input_shape[0], 1, 1)
        return memory

    def forward(self, input_ids, memory_state=None, **kwargs):
        if memory_state is None:
            memory_state = self.set_memory(input_ids.shape)

        seg_kwargs = self.process_input(input_ids, memory_state, **kwargs)
        out = self.model(**seg_kwargs)
        out, new_memory_state = self.process_output(out, **kwargs)

        return out, new_memory_state
    
    def generate(self, input_ids, memory_state, attention_mask, **generate_kwargs):
        if memory_state is None:
            memory_state = self.set_memory(input_ids.shape)

        seg_kwargs = self.process_input(input_ids, memory_state, attention_mask=attention_mask)
        out = self.model.generate(inputs_embeds=seg_kwargs['inputs_embeds'], attention_mask=seg_kwargs['attention_mask'], **generate_kwargs)
        return out

    def process_input(self, input_ids, memory_state, **kwargs):
        seg_kwargs = dict(**kwargs)

        inputs_embeds = kwargs.get('inputs_embeds')
        if inputs_embeds is None:
            inputs_embeds = self.model.get_input_embeddings()(input_ids)
        inputs_embeds = torch.cat([memory_state, inputs_embeds, memory_state], dim=1)

        seg_kwargs['input_ids'] = None
        seg_kwargs['inputs_embeds'] = inputs_embeds
        if kwargs.get('attention_mask') is not None:
            seg_kwargs['attention_mask'] = self.pad_attention_mask(kwargs['attention_mask'], inputs_embeds.shape)
        seg_kwargs['output_hidden_states'] = True
        return seg_kwargs
    
    def pad_attention_mask(self, attention_mask, shape):
        if self.num_mem_tokens in {0, None}:
            return attention_mask
        else:
            mask = torch.ones(*shape[:2], dtype=torch.int64).to(attention_mask.device)
            mask[:, self.num_mem_tokens:-self.num_mem_tokens] = attention_mask
            return mask
    
    def process_output(self, model_outputs, **kwargs):
        if self.num_mem_tokens not in {0, None}:
            out = CausalLMOutputWithCrossAttentions()
            memory_state = model_outputs.hidden_states[-1][:, -self.num_mem_tokens:]
            out['logits'] = model_outputs.logits[:, self.num_mem_tokens:-self.num_mem_tokens]
            
            if kwargs.get('output_hidden_states'):
                out['hidden_states'] = [lh[:, self.num_mem_tokens:-self.num_mem_tokens] for lh in model_outputs.hidden_states]
            if kwargs.get('output_attentions'):
                out['attentions'] = model_outputs['attentions']
        else:
            memory_state = None
            out = model_outputs
            
        return out, memory_state 


import random
class RecurrentWrapper(torch.nn.Module):
    def __init__(self, memory_cell, **rmt_kwargs):
        super().__init__()
        self.memory_cell = memory_cell
        self.rmt_config = rmt_kwargs

    def forward(self, input_ids, labels=None, labels_mask=None, inputs_embeds=None, attention_mask=None, output_attentions=None, output_hidden_states=None):
        memory_state = None
        segmented = self.segment(input_ids=input_ids, inputs_embeds=inputs_embeds, attention_mask=attention_mask)

        cell_outputs = []
        for seg_num, segment in enumerate(segmented):
            cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)
            cell_outputs.append(cell_out)
            self.manage_gradients(memory_state, seg_num)

        out = self.process_outputs(cell_outputs, labels=labels, 
                                   labels_mask=labels_mask,
                                   output_attentions=output_attentions, 
                                   output_hidden_states=output_hidden_states)
        return out
    
    def generate(self, input_ids, attention_mask, **generate_kwargs):
        memory_state = None
        segmented = self.segment(input_ids=input_ids, attention_mask=attention_mask)

        for seg_num, segment in enumerate(segmented[:-1]):
            cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)

        final_segment = segmented[-1]
        out = self.memory_cell.generate(**final_segment, memory_state=memory_state, **generate_kwargs)

        return out

    def segment(self, **kwargs):
        segments = []
        for k, tensor in kwargs.items():
            if tensor is not None:
                k_segments = self.split_tensor(tensor)
                for s, k_seg in enumerate(k_segments):
                    if s < len(segments):
                        segments[s][k] = k_seg
                    else:
                        segments.append({k: k_seg})
        
        max_n_segments = self.rmt_config.get('max_n_segments', False)
        if max_n_segments:
            # if self.rmt_config.get('vary_n_segments', False):
            #     max_n_segments = random.randint(1, max_n_segments+1)
            segments = segments[:max_n_segments]

        return segments
    
    def split_tensor(self, tensor):
        align = self.rmt_config.get('segment_alignment')
        segment_size = self.rmt_config.get('segment_size')
        if align in {'left', None}:
            split_inds = list(range(0, tensor.shape[1], segment_size)) + [tensor.shape[1]]
            segments = [tensor[:, start:end] for (start, end) in zip(split_inds, split_inds[1:])]
        elif align in {'right', None}:
            split_inds = (list(range(tensor.shape[1], 0, -segment_size)) + [0])[::-1]
            segments = [tensor[:, start:end] for (start, end) in zip(split_inds, split_inds[1:])]
        elif align == 'center':
            n_seg = math.ceil(tensor.shape[1] / segment_size)
            segments = torch.chunk(tensor, n_seg, dim=1)
        else:
            raise NotImplementedError
        return segments

    def process_outputs(self, cell_outputs, **kwargs):
        out = CausalLMOutputWithCrossAttentions()
        full_logits = torch.cat([o.logits for o in cell_outputs], dim=1)
        full_hidden_states = tuple([torch.cat(layer_hs, dim=1) for layer_hs in zip(*[o.hidden_states for o in cell_outputs])])

        labels = kwargs.get('labels')
        if labels is not None:
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = full_logits[..., :-1, :].contiguous()
            flat_labels = shift_labels.view(-1)
            flat_logits = shift_logits.view(-1, shift_logits.size(-1))
            
            loss_fct = CrossEntropyLoss()
            labels_mask = kwargs.get('labels_mask')
            # print('\n\n\n\n\n\n\n\n')
            # print('labels_mask', labels_mask.shape)
            # print('shift_logits', shift_logits.shape)
            # print('cell_outputs', len(cell_outputs))
            if labels_mask is not None:
                shift_mask = labels_mask[..., :-1].contiguous()

                flat_labels = flat_labels[shift_mask.view(-1)]
                flat_logits = flat_logits[shift_mask.view(-1)]
                
            out['loss'] = loss_fct(flat_logits, flat_labels)
        else:
            out['loss'] = 0

        out['logits'] = full_logits
        segment_keys = ['loss', 'logits']
        if kwargs.get('output_attentions'):
            segment_keys.append('attentions')
        if kwargs.get('output_hidden_states'):
            segment_keys.append('hidden_states')
            out['hidden_states'] = full_hidden_states

        for seg_num, o in enumerate(cell_outputs):
            for key, value in o.items():
                if any([sk in key for sk in segment_keys]):
                    out[f'{key}_{seg_num}'] = value

        return out 
        
    def manage_gradients(self, memory_state, seg_num):
        k2, max_n_segments = self.rmt_config.get('k2'), self.rmt_config.get('max_n_segments')
        if seg_num == 0 \
            or k2 in {-1, None} \
            or seg_num + k2 > max_n_segments:
                return True
        
        memory_state = memory_state.detach()
        return False

In [14]:
# from base_models.modeling_gpt_neox import GPTNeoXForCausalLM
# base_model = GPTNeoXForCausalLM.from_pretrained("EleutherAI/pythia-70m-deduped")

In [15]:
# base_model

In [44]:
import torch
from typing import Union, Optional, Tuple
from transformers.modeling_outputs import BaseModelOutputWithPast

def gpt_neox_horizontal_memory_forward(
    self,
    input_ids: Optional[torch.LongTensor] = None,
    attention_mask: Optional[torch.FloatTensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    head_mask: Optional[torch.FloatTensor] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
    use_cache: Optional[bool] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
    rmt_parent=None
) -> Union[Tuple, BaseModelOutputWithPast]:
    r"""
    past_key_values (`tuple(tuple(torch.FloatTensor))` of length `config.n_layers` with each tuple having 4 tensors of shape `(batch_size, num_heads, sequence_length - 1, embed_size_per_head)`):
        Contains precomputed key and value hidden states of the attention blocks. Can be used to speed up decoding.
        If `past_key_values` are used, the user can optionally input only the last `decoder_input_ids` (those that
        don't have their past key value states given to this model) of shape `(batch_size, 1)` instead of all
        `decoder_input_ids` of shape `(batch_size, sequence_length)`.
    use_cache (`bool`, *optional*):
        If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding (see
        `past_key_values`).
    """
    output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
    output_hidden_states = (
        output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
    )
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict
    use_cache = use_cache if use_cache is not None else self.config.use_cache

    if input_ids is not None and inputs_embeds is not None:
        raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
    elif input_ids is not None:
        input_shape = input_ids.size()
    elif inputs_embeds is not None:
        input_shape = inputs_embeds.size()[:-1]
    else:
        raise ValueError("You have to specify either input_ids or inputs_embeds")

    batch_size, seq_length = input_shape

    if past_key_values is None:
        past_length = 0
        past_key_values = tuple([None] * self.config.num_hidden_layers)
    else:
        past_length = past_key_values[0][0].size(-2)

    if position_ids is None:
        device = input_ids.device if input_ids is not None else inputs_embeds.device
        position_ids = torch.arange(past_length, seq_length + past_length, dtype=torch.long, device=device)
        position_ids = position_ids.unsqueeze(0).view(-1, seq_length)
    else:
        position_ids = position_ids.view(-1, seq_length).long()

    # Attention mask.
    if attention_mask is not None:
        assert batch_size > 0, "batch_size has to be defined and > 0"
        attention_mask = attention_mask.view(batch_size, -1)
        # We create a 3D attention mask from a 2D tensor mask.
        # Sizes are [batch_size, 1, 1, to_seq_length]
        # So we can broadcast to [batch_size, num_heads, from_seq_length, to_seq_length]
        # this attention mask is more simple than the triangular masking of causal attention
        # used in OpenAI GPT, we just need to prepare the broadcast dimension here.
        attention_mask = attention_mask[:, None, None, :]

        # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
        # masked positions, this operation will create a tensor which is 0.0 for
        # positions we want to attend and the dtype's smallest value for masked positions.
        # Since we are adding it to the raw scores before the softmax, this is
        # effectively the same as removing these entirely.
        attention_mask = attention_mask.to(dtype=self.dtype)  # fp16 compatibility
        attention_mask = (1.0 - attention_mask) * torch.finfo(self.dtype).min

    # Prepare head mask if needed
    # 1.0 in head_mask indicate we keep the head
    # attention_probs has shape bsz x n_heads x N x N
    # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
    # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
    head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

    if inputs_embeds is None:
        inputs_embeds = self.embed_in(input_ids)

    hidden_states = self.emb_dropout(inputs_embeds)

    if self.gradient_checkpointing and self.training:
        if use_cache:
            # logger.warning(
            #     "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
            # )
            use_cache = False

    presents = () if use_cache else None
    all_attentions = () if output_attentions else None
    all_hidden_states = () if output_hidden_states else None
    for i, (layer, layer_past) in enumerate(zip(self.layers, past_key_values)):
        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if self.gradient_checkpointing and self.training:
            raise NotImplementedError
            # def create_custom_forward(module):
            #     def custom_forward(*inputs):
            #         # None for layer_past
            #         return module(*inputs, use_cache, None, output_attentions)

            #     return custom_forward

            # outputs = torch.utils.checkpoint.checkpoint(
            #     create_custom_forward(layer),
            #     hidden_states,
            #     attention_mask,
            #     position_ids,
            #     head_mask[i],
            # )
        else:
            num_mem_tokens = rmt_parent.memory_cell.num_mem_tokens
            if i in rmt_parent.memory_storage:
                layer_memory = rmt_parent.memory_storage[i]
                if layer_memory.shape[0] == 1:
                    layer_memory = layer_memory.repeat(hidden_states.shape[0], 1, 1)

                hidden_states[:, :num_mem_tokens] = layer_memory
                    
            outputs = layer(
                hidden_states,
                attention_mask=attention_mask,
                position_ids=position_ids,
                head_mask=head_mask[i],
                layer_past=layer_past,
                use_cache=use_cache,
                output_attentions=output_attentions,
            )
        hidden_states = outputs[0]
        if use_cache is True:
            presents = presents + (outputs[1],)
        if output_attentions:
            all_attentions = all_attentions + (outputs[2 if use_cache else 1],)


    hidden_states = self.final_layer_norm(hidden_states)

    ### set layer memory
    rmt_parent.memory_storage[i] = hidden_states[:, -num_mem_tokens:].detach()

    # Add last hidden state
    if output_hidden_states:
        all_hidden_states = all_hidden_states + (hidden_states,)

    if not return_dict:
        return tuple(v for v in [hidden_states, presents, all_hidden_states, all_attentions] if v is not None)

    return BaseModelOutputWithPast(
        last_hidden_state=hidden_states,
        past_key_values=presents,
        hidden_states=all_hidden_states,
        attentions=all_attentions,
    )

In [54]:
raise ValueError("Hello")

ValueError: Hello

In [45]:
# def add_memory_layers(self):
#     memory_layers, share_memory_layers = self.rmt_config.get('memory_layers'), self.rmt_config.get('share_memory_layers')
#     if memory_layers is None:
#         self.memory_layers = None
#     else:
#         if memory_layers == 'all':
#             memory_layers = range(len(self.model.base_model.encoder.layer))
#         else:
#             raise NotImplementedError
            
#         if share_memory_layers:
#             memory_layer = copy.deepcopy(self.model.base_model.encoder.layer[0])
#             self.memory_layers = [memory_layer for _ in range(len(memory_layers))]
#             for n, p in memory_layer.named_parameters():
#                 param_name = re.sub('\.', '_', f'memory_{n}')
#                 self.register_parameter(param_name, p)
#         else:
#             self.memory_layers = [copy.deepcopy(self.model.base_model.encoder.layer[int(l)]) for l in memory_layers]
#             for ln, layer in enumerate(self.memory_layers):
#                 for n, p in layer.named_parameters():
#                     param_name = re.sub('\.', '_', f'{ln}_memory_{n}')
#                     self.register_parameter(param_name, p)


#                     add_memory_layers()
    
# # memory_forward_func = rmt_config.get('memory_forward_func')
# # if memory_forward_func is None:
# #     memory_forward_func = horizontal_memory_forward
# # self.override_encoder_forward(memory_forward_func)

In [46]:
block_size=args.input_size-2*args.num_mem_tokens
# define model
model_cls = get_cls_by_name(args.model_cls)
# if hvd.rank() == 0:
#     logger.info(f'Using model class: {model_cls}')
if not args.from_pretrained:
    model_cfg = AutoConfig.from_pretrained(args.model_cfg)
    model = model_cls(config=model_cfg)
else:
    # if hvd.rank() == 0:
    #     logger.info(f'Loading pretrained model: {args.from_pretrained}')
    model = model_cls.from_pretrained(args.from_pretrained)

model.resize_token_embeddings(len(tokenizer))

## load cpt of backbone model
if args.backbone_cpt:
    backbone_cpt = os.path.join(args.backbone_cpt, "model_best.pth")
    cpt = torch.load(backbone_cpt, map_location='cpu')
    model.load_state_dict(cpt['model_state_dict'], strict=False)
    # if hvd.rank() == 0:
    #     logger.info(f'Loaded baseline state dict from: {args.backbone_cpt}')

# Pass memory settings to pretrained model
if args.num_mem_tokens is not None:
    # memory_cell_cls = get_cls_by_name(args.memory_cell_cls)
    # recurrent_wrapper_cls = get_cls_by_name(args.recurrent_wrapper_cls)
    memory_cell_cls = MemoryCell
    recurrent_wrapper_cls = RecurrentWrapper
    # if hvd.rank() == 0:
    #     logger.info(f'Wrapping in: {memory_cell_cls} and {recurrent_wrapper_cls}')
    
    
    cell = memory_cell_cls(model, args.num_mem_tokens)
    model = recurrent_wrapper_cls(cell, 
                                    segment_size=block_size,
                                    max_n_segments=args.max_n_segments, 
                                    k2=args.k2,
                                    segment_alignment=args.segment_alignment
    )
                                

    ## load cpt of rmt
    if args.model_cpt:
        model_cpt = os.path.join(args.model_cpt, "model_best.pth")
        cpt = torch.load(model_cpt, map_location='cpu')
        model.load_state_dict(cpt['model_state_dict'], strict=False)
        # if hvd.rank() == 0:
            # logger.info(f'Loaded RMT state dict from: {args.model_cpt}')
        print(f'Loaded RMT state dict from: {args.model_cpt}')

if args.freeze_model_weights:
    for n, p in model.named_parameters():
        # if 'memory' not in n and 'wte' not in n:
        if 'memory' not in n and 'lora' not in n:
            p.requires_grad = False
    # if hvd.rank() == 0:
    #     logger.info(f'Frozen moodel weights')
    #     logger.info(f'Remaining parameters: {[n for n, p in model.named_parameters() if p.requires_grad]}')

# # fix the not-contiguous error with loralib and horovod
# def make_contiguous(module):
#     with torch.no_grad():
#         for param in module.parameters():
#             param.set_(param.contiguous())
# make_contiguous(model)


loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--EleutherAI--pythia-70m-deduped/snapshots/e93a9faa9c77e5d09219f6c868bfc7a1bd65593c/config.json
Model config GPTNeoXConfig {
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 8,
  "num_hidden_layers": 6,
  "rope_scaling": null,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.25,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "use_parallel_residual": true,
  "vocab_size": 50304
}

loading weights file model.safetensors from cache at /home/jovyan/.cache/huggingface/hub/models--EleutherAI-

In [47]:
# def override_encoder_forward(memory_forward_func)
import types

self = model
new_forward = lambda *args, **kwargs: gpt_neox_horizontal_memory_forward(*args, **kwargs, rmt_parent=self)
self.memory_cell.model.gpt_neox.forward = types.MethodType(new_forward, self.memory_cell.model.gpt_neox)

In [48]:

# define optimizer
optimizer_cls = get_optimizer(args.optimizer)
if optimizer_cls is None:
    raise RuntimeError(f'{args.optimizer} was not found in optimizers, torch.optim, transformers.optimization')

# if hvd.rank() == 0:
#     logger.info(f'Using optimizer class: {optimizer_cls}')

# todo: group optimizer params
if optimizer_cls in [transformers.optimization.Adafactor, optimizers.Adafactor]:
    # https://github.com/huggingface/transformers/pull/9751/files -> transformers 4.3.0
    optimizer = optimizer_cls(model.parameters(), lr=args.lr,
                                scale_parameter=args.scale_parameter,
                                relative_step=args.relative_step,
                                warmup_init=args.warmup_init,
                                weight_decay=args.weight_decay)
else:
    optimizer = optimizer_cls(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

def keep_for_metrics_fn(batch, output):
    data = {}
    if 'generation_outputs' in output:
            # data['labels'] = batch['answer']
        data['generation_outputs'] = output['generation_outputs']
        
    data['labels'] = batch['labels']
    for key in batch.keys():
        if 'loss' in key: 
            data[key] = batch[key]
    data['predictions'] = torch.argmax(output['logits'].detach(), dim=-1)
    if 'labels_mask' in batch:
        data['predictions'] = [data['predictions'][i, mask] for i, mask in enumerate(batch['labels_mask'])]
    return data

# HF datasets can compute metrics on each gpu process and then aggregate them on process with rank 0
# synchronization is done by using temporay files on a shared filesystem
# rank and number of workers is set by num_process and process_id params
# BUT our Trainer aggregates all prediction from all gpus!
#   this will lead to computing metrics for predictions repeated xN_GPUS times
# need to try:
# - keep_in_memory=True, may lead to OOM for large validation sets, after sync predictions and targets for the full
#       validation set would be stored on each GPU -> xN_GPUs RAM
#   - implemented currently
# - compute metrics on batch lvl
# - add support of HF metrics and turn off aggregation in case if metric has .add_batch method
# scrolls_metric = datasets.load_metric(scrolls_metric_path, args.task_name, keep_in_memory=True)

def metrics_fn(data):
    metrics = {}
    y, p = None, None
    if 'generation_outputs' in data:
        # replace -100 with pad token in labels
        y = data['labels']
        p = tokenizer.batch_decode(data['generation_outputs'], skip_special_tokens=True)
        
        if hvd.rank() == 0 and args.show_valid_examples > 0:
            for i in range(min(args.show_valid_examples, len(y))):
                logger.info(f'y: {y[i]}')
                logger.info(f'p: {p[i]}')
                logger.info(f'p ids: {data["generation_outputs"][i]}')
                logger.info('-' * 50)

    if y is not None and p is not None:
        metrics['exact_match'] = accuracy_score(y, p) * 100

    return metrics


In [49]:
train_gen=iter(train_dataloader)
train_batch = next(train_gen)
train_batch.keys()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


dict_keys(['input_ids', 'labels', 'input_ids_generate', 'labels_mask', 'attention_mask', 'id', 'target_text'])

In [50]:
train_batch.pop('id')
train_batch.pop('target_text')
train_batch.pop('input_ids_generate')

tensor([[ 1785, 32177,  7021,  3091,   417,  8107, 16518,   667,  5113,   534,
           802,  2915,  5201, 49209,  7021,   434, 11204,  9440,  8339,    15,
           187,   187, 34656,    14, 15857,  3207,  2697, 10860,  4889,  5836,
            39, 25788, 13859,  7400, 13145,  1848, 17260,   187,  1552, 36429,
            14, 15857,  3207,  2697, 10860,  4889,  5836,    39, 25788, 13859,
          7400, 13145,  1848, 17260, 11126,  8903, 10742, 15440,   310,  1160,
           407,   285,   875,    27,   187,     9,    74,    10,   253,  7454,
           273,   253,  1986, 14726,  4855, 15605,   323, 44438,   265,    13,
          1907,   697, 17929,  4441,   387, 11107, 43744,   372,  7812,  1288,
           408,   386,    13,  7346,    19, 31955,    13, 18908,   313, 30350,
           773,  4037,    41,  1311,   668,   390,   253,   773,  6744,   498,
         14356,   668,   558,   285,   187,     9,  2886,    10, 27103,  1713,
         15362,  1157,   247,  2567,  4232,   275,  

In [52]:
model.memory_storage = {}

In [53]:

out = model(**train_batch)
out.loss

entering custom forward
entering custom forward


tensor(8.5803, grad_fn=<NllLossBackward0>)

In [92]:
train_batch['attention_mask'].shape

torch.Size([2, 177])

In [97]:
model.rmt_config['segment_alignment'] = 'center'
segmented = model.segment(input_ids=train_batch['input_ids'])

In [98]:
model.rmt_config

{'segment_size': 118,
 'max_n_segments': 2,
 'k2': -1,
 'segment_alignment': 'center'}

In [100]:
segmented[0]['input_ids'].shape, segmented[1]['input_ids'].shape

(torch.Size([2, 89]), torch.Size([2, 88]))

In [73]:
batch = next(train_gen)
batch.keys()

dict_keys(['input_ids', 'labels', 'input_ids_generate', 'labels_mask', 'attention_mask', 'id', 'target_text'])

In [74]:
# input_ids_generate = batch['input_ids_generate']
# attention_mask = batch['attention_mask']
# attention_mask = input_ids_generate != tokenizer.pad_token_id
# gen_out = model.generate(input_ids_generate, attention_mask=attention_mask, output_scores=True, return_dict_in_generate=True, max_new_tokens=30)

In [75]:
# ef forward(self, input_ids, labels=None, labels_mask=None, inputs_embeds=None, attention_mask=None, output_attentions=None, output_hidden_states=None):
self = model
input_ids = batch['input_ids']
input_ids_generate = batch['input_ids_generate']
attention_mask = batch['attention_mask']
labels = batch['labels']
labels_mask = batch['labels_mask']

inputs_embeds = None
output_attentions = False
output_hidden_states = False

memory_state = None
segmented = self.segment(input_ids=input_ids, inputs_embeds=inputs_embeds, attention_mask=attention_mask)

cell_outputs = []
for seg_num, segment in enumerate(segmented):
    cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)
    cell_outputs.append(cell_out)
    self.manage_gradients(memory_state, seg_num)

out = self.process_outputs(cell_outputs, labels=labels, 
                            labels_mask=labels_mask,
                            output_attentions=output_attentions, 
                            output_hidden_states=output_hidden_states)
# return out


In [76]:
len(segmented)

1

In [77]:
out.loss

tensor(0.1700, grad_fn=<NllLossBackward0>)

In [78]:
# def generate(self, input_ids, attention_mask, **generate_kwargs):
# generate_kwargs = {'output_scores': True, 'return_dict_in_generate': True}
generate_kwargs = {}

memory_state = None
segmented = self.segment(input_ids=input_ids_generate, attention_mask=attention_mask)

# cell_outputs = []
for seg_num, segment in enumerate(segmented[:-1]):
    print('forwarding')
    cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)
    # cell_outputs.append(cell_out)
    # self.manage_gradients(memory_state, seg_num)

final_segment = segmented[-1]
gen_out = self.memory_cell.generate(**final_segment, memory_state=memory_state, **generate_kwargs)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [89]:
predictions = tokenizer.batch_decode(gen_out[:, :], skip_special_tokens=True)

In [90]:
labels = batch['target_text']

In [91]:
predictions

['EntailmentNot mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned',
 'Not mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned']

In [92]:
labels

['Entailment', 'Entailment']

In [93]:
[l == p[:len(l)] for p, l in zip (predictions, labels)]

[True, False]

In [115]:
predictions

['Not mentioned mentioned mentioned mentionedNot mentioned mentioned mentioned mentionedNot mentioned mentioned mentioned mentioned mentionedNot mentioned mentioned',
 'Entailment mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned']

In [95]:
gen_out.sequences

tensor([[50256,  3673,  4750,  4750,  4750,  4750,  3673,  4750,  4750,  4750,
          4750,  3673,  4750,  4750,  4750,  4750,  4750,  3673,  4750,  4750],
        [50256, 14539,   603,   434,  4750,  4750,  4750,  4750,  4750,  4750,
          4750,  4750,  4750,  4750,  4750,  4750,  4750,  4750,  4750,  4750]])

In [81]:

input_ids[:, -1]

tensor([4750,  434])

In [50]:
tokenizer.batch_decode(input_ids)

["Receiving Party shall not reverse engineer any objects which embody Disclosing Party's Confidential Information.\n\nNON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT\nThis NON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT (“Agreement”) is made by and between:\n(i) the Office of the United Nations High Commissioner for Refugees, having its headquarters located at 94 rue de Montbrillant, 1202 Geneva, Switzerland (hereinafter “UNHCR” or the “Discloser[GEN]Not mentioned",
 'Receiving Party shall destroy or return some Confidential Information upon the termination of Agreement.\n\nNON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT\nThis NON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT (“Agreement”) is made by and between:\n(i) the Office of the United Nations High Commissioner for Refugees, having its headquarters located at 94 rue de Montbrillant, 1202 Geneva, Switzerland (hereinafter “UNHCR” or the “Discloser”[GEN]Entailment']

In [74]:
tokenizer.batch_decode(input_ids_generate[:1, :-2])

["Receiving Party shall not reverse engineer any objects which embody Disclosing Party's Confidential Information.\n\nNON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT\nThis NON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT (“Agreement”) is made by and between:\n(i) the Office of the United Nations High Commissioner for Refugees, having its headquarters located at 94 rue de Montbrillant, 1202 Geneva, Switzerland (hereinafter “UNHCR” or the “Discloser[GEN]"]

In [80]:
# model.generate(input_ids_generate[:1, :-2], attention_mask=torch.ones_like(input_ids_generate[:1, :-2]).bool())

In [69]:
attention_mask[0]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True])

In [68]:
# tokenizer.batch_decode(input_ids_generate[0][attention_mask[0]])

In [57]:
preds = out.logits.argmax(dim=-1)

In [58]:
tokenizer.batch_decode(preds[:, -10:])

['NotNot�NotclosureosingmentNot mentioned mentioned',
 'Entclosureailment�EntEntailment mentioned']

In [60]:
tokenizer.batch_decode(preds[0][labels_mask[0]][-10:])

['Not', ' mentioned', ' mentioned']

In [61]:
tokenizer.batch_decode(labels[0][labels_mask[0]][-10:])

['[GEN]', 'Not', ' mentioned']

In [63]:
tokenizer.batch_decode(labels[0][labels_mask[0]])

['[GEN]', 'Not', ' mentioned']

In [ ]:
valid_gen=iter(train_dataloader)
valid_batch = next(valid_gen)
valid_batch.keys()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


dict_keys(['input_ids', 'labels', 'input_ids_generate', 'labels_mask', 'attention_mask', 'id', 'target_text'])

In [198]:
input_ids = valid_batch['input_ids']

In [199]:
out = model(input_ids, labels=valid_batch['labels'], labels_mask=valid_batch['labels_mask'])

In [200]:
preds = out.logits.argmax(dim=-1)

In [201]:
[tokenizer.batch_decode(preds[i][valid_batch['labels_mask'][i]]) for i in range(2)]

[['Not', ' mentioned', ' mentioned'], ['Ent', 'ail', 'ment', ' mentioned']]

In [202]:
[tokenizer.batch_decode(input_ids[i][valid_batch['labels_mask'][i]]) for i in range(2)]

[['[GEN]', 'Not', ' mentioned'], ['[GEN]', 'Ent', 'ail', 'ment']]

In [204]:
gen_text = tokenizer.batch_decode(gen_out.sequences)
gen_text

['<|endoftext|>Not mentioned mentioned mentioned mentionedNot mentioned mentioned mentioned mentionedNot mentioned mentioned mentioned mentioned mentionedNot mentioned mentioned mentioned mentioned mentioned mentionedNot mentioned mentioned mentioned mentioned mentioned mentioned',
 '<|endoftext|>Entailment mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned mentioned']

In [205]:
tokenizer.batch_decode(input_ids)tokenizer.batch_decode(input_ids)

["Receiving Party shall not reverse engineer any objects which embody Disclosing Party's Confidential Information.\n\nNON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT\nThis NON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT (“Agreement”) is made by and between:\n(i) the Office of the United Nations High Commissioner for Refugees, having its headquarters located at 94 rue de Montbrillant, 1202 Geneva, Switzerland (hereinafter “UNHCR” or the “Discloser[GEN]Not mentioned",
 'Receiving Party shall destroy or return some Confidential Information upon the termination of Agreement.\n\nNON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT\nThis NON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT (“Agreement”) is made by and between:\n(i) the Office of the United Nations High Commissioner for Refugees, having its headquarters located at 94 rue de Montbrillant, 1202 Geneva, Switzerland (hereinafter “UNHCR” or the “Discloser”[GEN]Entailment']

In [206]:
tokenizer.batch_decode(input_ids_generate)

["Receiving Party shall not reverse engineer any objects which embody Disclosing Party's Confidential Information.\n\nNON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT\nThis NON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT (“Agreement”) is made by and between:\n(i) the Office of the United Nations High Commissioner for Refugees, having its headquarters located at 94 rue de Montbrillant, 1202 Geneva, Switzerland (hereinafter “UNHCR” or the “Discloser[GEN][PAD][PAD]",
 'Receiving Party shall destroy or return some Confidential Information upon the termination of Agreement.\n\nNON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT\nThis NON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT (“Agreement”) is made by and between:\n(i) the Office of the United Nations High Commissioner for Refugees, having its headquarters located at 94 rue de Montbrillant, 1202 Geneva, Switzerland (hereinafter “UNHCR” or the “Discloser”[GEN][PAD][PAD][PAD]']

In [212]:
tokenizer.decode(preds[0])

'[GEN][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]Not\nNotailNotNotCLosingURENotNotFLIDENTIALNotNotRE[GEN]Not[PAD] mentioned-NotCLOSURE and CON[PAD][PAD][PAD]ITY Agreement[PAD]EMENT shallNot�Notreement mentioned�EntNot mentionedNotNotNotNotNot[GEN]Ent)NotNot of the Director States, Commissioner for[PAD]NotNot mentionedEnt located in:-ue[GEN][PAD][PAD][PAD][PAD],[PAD][PAD][PAD], FranceEntEnt mentionedfter mentioned�EntNotEnt�EntNotNot�NotclosureosingmentNot mentioned mentioned'

In [217]:
optim = torch.optim.Adam(model.parameters())

In [218]:
optim.load_state_dict(cpt['optimizer_state_dict'])

In [214]:
cpt.keys()

dict_keys(['model_state_dict', 'optimizer_state_dict', 'iteration', 'epoch', 'metrics', 'lr_scheduler_state_dict'])

In [210]:
gen_out.sequences[0]

tensor([50256,  3673,  4750,  4750,  4750,  4750,  3673,  4750,  4750,  4750,
         4750,  3673,  4750,  4750,  4750,  4750,  4750,  3673,  4750,  4750,
         4750,  4750,  4750,  4750,  3673,  4750,  4750,  4750,  4750,  4750,
         4750])

In [208]:
input_ids_generate[0][attention_mask[0]]

tensor([ 3041,   344,  1412,  3615,  2236,   407,  9575, 11949,   597,  5563,
          543, 48061,  3167,   565,  2752,  3615,   338,  7326, 35599,  6188,
           13,   198,   198,    45,  1340,    12, 26288,  5097,  2640, 11335,
         5357,  7102,    37, 25256, 12576,  9050, 13077,  2200, 12529,   198,
         1212, 44521,    12, 26288,  5097,  2640, 11335,  5357,  7102,    37,
        25256, 12576,  9050, 13077,  2200, 12529,   357,   447,   250, 10262,
        10237,   447,   251,     8,   318,   925,   416,   290,  1022,    25,
          198,     7,    72,     8,   262,  4452,   286,   262,  1578,  7973,
         3334, 13270,   329, 38550,    11,  1719,   663, 10043,  5140,   379,
        10048,   374,   518,   390,  5575,  1671,   359,   415,    11,  1105,
         2999, 20552,    11, 14679,   357,  1456,  1437,   637,   564,   250,
         4944, 43230,   447,   251,   393,   262,   564,   250,  7279,   565,
        13416, 50257])

In [209]:
input_ids[0][attention_mask[0]]

tensor([ 3041,   344,  1412,  3615,  2236,   407,  9575, 11949,   597,  5563,
          543, 48061,  3167,   565,  2752,  3615,   338,  7326, 35599,  6188,
           13,   198,   198,    45,  1340,    12, 26288,  5097,  2640, 11335,
         5357,  7102,    37, 25256, 12576,  9050, 13077,  2200, 12529,   198,
         1212, 44521,    12, 26288,  5097,  2640, 11335,  5357,  7102,    37,
        25256, 12576,  9050, 13077,  2200, 12529,   357,   447,   250, 10262,
        10237,   447,   251,     8,   318,   925,   416,   290,  1022,    25,
          198,     7,    72,     8,   262,  4452,   286,   262,  1578,  7973,
         3334, 13270,   329, 38550,    11,  1719,   663, 10043,  5140,   379,
        10048,   374,   518,   390,  5575,  1671,   359,   415,    11,  1105,
         2999, 20552,    11, 14679,   357,  1456,  1437,   637,   564,   250,
         4944, 43230,   447,   251,   393,   262,   564,   250,  7279,   565,
        13416, 50257])

In [82]:
from transformers import GPT2LMHeadModel, AutoTokenizer
gpt = GPT2LMHeadModel.from_pretrained('gpt2')
gpt_tok = AutoTokenizer.from_pretrained('gpt2')

In [91]:
input_ids = gpt_tok.batch_encode_plus(['The meaning of life is'], return_tensors='pt')
gpt_out = gpt.generate(**input_ids, output_scores=True, return_dict_in_generate=True,)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [92]:
input_ids

{'input_ids': tensor([[ 464, 3616,  286, 1204,  318]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [100]:
# gpt

In [102]:
inputs_embeds = gpt.transformer.wte(input_ids['input_ids'])

In [103]:
inputs_embeds.shape

torch.Size([1, 5, 768])

In [104]:
gpt_out = gpt.generate(inputs_embeds=inputs_embeds)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [106]:
gpt_out

tensor([[50256,   407,   262,   976,   355,   262,  3616,   286,  1918,    13,
           198,   198,   464,  3616,   286,  1204,   318,   407,   262,   976]])

In [108]:
tokenizer.decode(gpt_out[0])

'<|endoftext|> not the same as the meaning of death.\n\nThe meaning of life is not the same'

In [ ]:
tokenizer.decode(gpt_out.sequences[0])

'The meaning of life is not the same as the meaning of death.\n\nThe meaning of life'

In [ ]:
tokenizer.decode([])

'I love cats. I love cats. I love cats. I love cats. I love cats.'

In [105]:
len(gpt_out.scores), gpt_out.scores[0].shape

(17, torch.Size([1, 50257]))

In [81]:
valid_batch['input_ids'].shape

torch.Size([2, 236])

In [106]:
gen_out = model.generate(**valid_batch, output_scores=True, return_dict_in_generate=True)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [123]:
hasattr(gen_out, 'sequences')

True

In [74]:
for k in train_batch:
    print(k, train_batch[k].shape)

input_ids torch.Size([2, 236])
labels torch.Size([2, 236])
labels_mask torch.Size([2, 236])
attention_mask torch.Size([2, 236])


In [50]:
segs[0]['input_ids'].shape

torch.Size([2, 118])

In [46]:
segs = self.segment(input_ids=input_ids)
len(segs)

1

In [132]:
# out = model(**train_batch)
# out.loss

self = model
input_ids = train_batch['input_ids']
inputs_embeds = None
attention_mask = train_batch['attention_mask']

memory_state = None
segmented = self.segment(input_ids=input_ids, inputs_embeds=inputs_embeds, attention_mask=attention_mask)

cell_outputs = []
for seg_num, segment in enumerate(segmented):
    cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)
    cell_outputs.append(cell_out)
    self.manage_gradients(memory_state, seg_num)

In [133]:
len(segmented)

1

In [136]:
segmented[0]['attention_mask'].shape

torch.Size([2, 124])

In [38]:
cell_outputs[0]

IndexError: list index out of range

In [ ]:
out = model(**train_batch)
out.loss

tensor(96.6092, grad_fn=<NllLossBackward0>)

In [ ]:
out = model(**train_batch)
out.loss

tensor(84.5565, grad_fn=<NllLossBackward0>)

In [ ]:
out = model(**train_batch)
out.loss

tensor(86.7867, grad_fn=<NllLossBackward0>)

In [35]:
preds = tokenizer.batch_decode(gen_out, skip_special_tokens=False)

In [39]:
eos = tokenizer.eos_token
[p[:p.index(eos)] if eos in p else p for p in preds]

['', '']

In [31]:
(torch.eye(5) == 0).nonzero(as_tuple=True)

(tensor([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]),
 tensor([1, 2, 3, 4, 0, 2, 3, 4, 0, 1, 3, 4, 0, 1, 2, 4, 0, 1, 2, 3]))

In [32]:
(torch.eye(5) == 0).nonzero(as_tuple=False)

tensor([[0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [1, 0],
        [1, 2],
        [1, 3],
        [1, 4],
        [2, 0],
        [2, 1],
        [2, 3],
        [2, 4],
        [3, 0],
        [3, 1],
        [3, 2],
        [3, 4],
        [4, 0],
        [4, 1],
        [4, 2],
        [4, 3]])

In [26]:
(gen_out == tokenizer.eos_token_id).nonzero(as_tuple=True)

(tensor([0, 1]), tensor([0, 0]))

In [27]:
(gen_out == tokenizer.eos_token_id).nonzero(as_tuple=False)

tensor([[0, 0],
        [1, 0]])

### Debug

In [50]:
# gen = model.generate(**valid_batch)
self = model
input_ids = valid_batch['input_ids']
attention_mask = valid_batch['attention_mask']


memory_state = None
segmented = self.segment(input_ids=input_ids, attention_mask=attention_mask)

# cell_outputs = []
for seg_num, segment in enumerate(segmented[:-1]):
    cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)
    # cell_outputs.append(cell_out)
    # self.manage_gradients(memory_state, seg_num)

final_segment = segmented[-1]
# out = self.memory_cell.generate(**final_segment, memory_state=memory_state)


In [63]:
self = model.memory_cell
input_ids = final_segment['input_ids']
attention_mask = final_segment['attention_mask']
generate_kwargs = {'max_new_tokens': 20}

# def generate(self, input_ids, memory_state, attention_mask, **generate_kwargs)
if memory_state is None:
    memory_state = self.set_memory(input_ids.shape)

seg_kwargs = self.process_input(input_ids, memory_state, attention_mask=attention_mask)
out = self.model.generate(inputs_embeds=seg_kwargs['inputs_embeds'], attention_mask=seg_kwargs['attention_mask'], **generate_kwargs)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [64]:
out.shape

torch.Size([2, 21])

In [65]:
tokenizer.batch_decode(out)

['<|endoftext|>[GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN]',
 '<|endoftext|>[GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN]']

In [ ]:
import math
from transformers.modeling_outputs import CausalLMOutputWithCrossAttentions

class MemoryCell(torch.nn.Module):
    def __init__(self, base_model, num_mem_tokens):
        super().__init__()
        self.model = base_model
        self.create_memory(num_mem_tokens)

    def create_memory(self, num_mem_tokens):
        self.num_mem_tokens = num_mem_tokens
        embeddings = self.model.get_input_embeddings()
        memory_weights = torch.randn((num_mem_tokens, self.model.config.n_embd)) * embeddings.weight.data.std()
        self.register_parameter('memory', torch.nn.Parameter(memory_weights, requires_grad=True))

        self.read_memory_position = range(num_mem_tokens)
        self.write_memory_position = range(-num_mem_tokens, 0)

    def set_memory(self, input_shape):
        memory = self.memory.repeat(input_shape[0], 1, 1)
        return memory

    def forward(self, input_ids, memory_state=None, **kwargs):
        if memory_state is None:
            memory_state = self.set_memory(input_ids.shape)

        seg_kwargs = self.process_input(input_ids, memory_state, **kwargs)
        out = self.model(**seg_kwargs)
        out, new_memory_state = self.process_output(out, **kwargs)

        return out, new_memory_state
    
    def generate(self, input_ids, memory_state, attention_mask, **generate_kwargs)
        if memory_state is None:
            memory_state = self.set_memory(input_ids.shape)

        seg_kwargs = self.process_input(input_ids, memory_state, attention_mask=attention_mask)
        out = self.model.generate(seg_kwargs.inputs_embeds, **generate_kwargs)
        return out
    
    def process_input(self, input_ids, memory_state, **kwargs):
        seg_kwargs = dict(**kwargs)

        inputs_embeds = kwargs.get('inputs_embeds')
        if inputs_embeds is None:
            inputs_embeds = self.model.get_input_embeddings()(input_ids)
        inputs_embeds = torch.cat([memory_state, inputs_embeds, memory_state], dim=1)

        seg_kwargs['input_ids'] = None
        seg_kwargs['inputs_embeds'] = inputs_embeds
        if kwargs.get('attention_mask') is not None:
            seg_kwargs['attention_mask'] = self.pad_attention_mask(kwargs['attention_mask'], inputs_embeds.shape)
        seg_kwargs['output_hidden_states'] = True
        return seg_kwargs
    
    def pad_attention_mask(self, attention_mask, shape):
        if self.num_mem_tokens in {0, None}:
            return attention_mask
        else:
            mask = torch.ones(*shape[:2], dtype=torch.int64).to(attention_mask.device)
            mask[:, self.num_mem_tokens:-self.num_mem_tokens] = attention_mask
            return mask
    
    def process_output(self, model_outputs, **kwargs):
        if self.num_mem_tokens not in {0, None}:
            out = CausalLMOutputWithCrossAttentions()
            memory_state = model_outputs.hidden_states[-1][:, -self.num_mem_tokens:]
            out['logits'] = model_outputs.logits[:, self.num_mem_tokens:-self.num_mem_tokens]
            
            if kwargs.get('output_hidden_states'):
                out['hidden_states'] = [lh[:, self.num_mem_tokens:-self.num_mem_tokens] for lh in model_outputs.hidden_states]
            if kwargs.get('output_attentions'):
                out['attentions'] = model_outputs['attentions']
        else:
            memory_state = None
            out = model_outputs
            
        return out, memory_state 

In [ ]:
tokenizer.decode(gen_inputs[0])

'Receiving Party shall not disclose the fact that Agreement was agreed or negotiated. Johns Hopkins University\nNON-DISCLOSURE AGREEMENT For Bilateral Disclosure\nThis Agreement is effective this of in the year ______ is by and between JHU and COMPANY, each defined below.\nJHU: The Johns Hopkins University\nAddress: 100 N. Charles St., 5th Floor\nBaltimore, Maryland 21201\nJHU Contact:\nCOMPANY:\nAddress:\nCOMPANY Contact:\nWHEREAS, each party has certain technical information described below which shall hereinafter be referred to as "CONFIDENTIAL INFORMATION";\nCONFIDENTIAL INFORMATION:\nWHEREAS, JHU and COMPANY are each interested in examining the CONFIDENTIAL INFORMATION of the other solely for the PURPOSE, defined below;\nPURPOSE: To explore licensing, collaborative or sponsored research agreement opportunities related to the CONFIDENTIAL INFORMATION.\nNOW, THEREFORE, in consideration of the premises and mutual covenants contained herein, the parties hereto agree as follows:\n”PROV

In [19]:
gt = tokenizer.batch_decode([l[m] for l, m in zip(batch['labels'], batch['labels_mask'])], skip_special_tokens=True)

In [20]:
gt

['Not mentioned', 'Entailment']

In [ ]:
accuracy_score(pred, pred)

1.0

In [ ]:
tokenizer.batch_decode([l[m] for l, m in zip(batch['labels'], batch['labels_mask'])], skip_special_tokens=True)

['NotMentioned', 'Contradiction']

In [ ]:
out.loss

tensor(62.5257, grad_fn=<NllLossBackward>)

In [ ]:
for i in range(15):
    out = model(**batch)
    out.loss.backward()
    optimizer.step()

In [ ]:
out.loss

tensor(7.8596, grad_fn=<NllLossBackward>)

In [ ]:
pred = out.logits.argmax(dim=-1)

In [ ]:
pred_text = tokenizer.batch_decode(pred)

In [ ]:
generated = model.memory_cell.model.generate(batch['labels'][:, -5:])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
batch['input_ids'][0][batch['labels_mask'][0]]

tensor([50257,  3673,    44,  1463,   276, 50256])

In [ ]:
batch['labels'][0][batch['labels_mask'][0]]

tensor([50257,  3673,    44,  1463,   276, 50256])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
output = generated

In [ ]:
 def keep_for_metrics_fn(batch, output):
    data = {}
    if isinstance(output, torch.Tensor):
        data['labels'] = batch['labels']
        
    for key in batch.keys():
        if 'loss' in key: 
            data[key] = batch[key]
    return data

In [ ]:
tokenizer.batch_decode(generated, add_special_tokens=False)

['[GEN]NotMentioned[GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN]',
 ' in[GEN]Contradiction[GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN]']

In [ ]:
#  def keep_for_metrics_fn(batch, output):
data = {}
# if 'generation_outputs' in output:
if isinstance(output, dict):
    data['predictions'] = torch.argmax(output['logits'].detach(), dim=-1)
    if 'labels_mask' in batch:
        data['predictions'] = [data['predictions'][i, mask] for i, mask in enumerate(batch['labels_mask'])]
else:
    data['generation_outputs'] = output
    
data['labels'] = batch['labels']
for key in batch.keys():
    if 'loss' in key: 
        data[key] = batch[key]

In [ ]:
# out = CausalLMOutputWithCrossAttentions()
# full_logits = torch.cat([o.logits for o in cell_outputs], dim=1)
# full_hidden_states = tuple([torch.cat(layer_hs, dim=1) for layer_hs in zip(*[o.hidden_states for o in cell_outputs])])

full_logits = out.logits
labels = kwargs.get('labels')
if labels is not None:
    shift_labels = labels[..., 1:].contiguous()
    shift_logits = full_logits[..., :-1, :].contiguous()
    flat_labels = shift_labels.view(-1)
    flat_logits = shift_logits.view(-1, shift_logits.size(-1))
    
    loss_fct = CrossEntropyLoss()
    labels_mask = kwargs.get('labels_mask')
    if labels_mask is not None:
        shift_mask = labels_mask[..., :-1].contiguous()

        flat_labels = flat_labels[shift_mask.view(-1)]
        flat_logits = flat_logits[shift_mask.view(-1)]
        
    out['loss'] = loss_fct(flat_logits, flat_labels)

out['logits'] = full_logits
segment_keys = ['loss', 'logits']
if kwargs.get('output_attentions'):
    segment_keys.append('attentions')
if kwargs.get('output_hidden_states'):
    segment_keys.append('hidden_states')
    out['hidden_states'] = full_hidden_states

In [ ]:

### booydar
batch_metrics_fn = lambda _, y: {key: y[key] for key in y.keys() if (('loss' in key) or ('!log' in key))}
trainer = Trainer(args, model, optimizer, train_dataloader, valid_dataloader, train_sampler,
                    keep_for_metrics_fn=keep_for_metrics_fn, metrics_fn=metrics_fn,
                    ###booydar
                    batch_metrics_fn=batch_metrics_fn,
                    generate_kwargs={})

if not args.validate_only:
    # train loop
    trainer.train()
    # make sure all workers are done
    hvd.barrier()
    # run validation after training
    if args.save_best:
        best_model_path = str(Path(args.model_path) / 'model_best.pth')
        if hvd.rank() == 0:
            logger.info(f'Loading best saved model from {best_model_path}')
        trainer.load(best_model_path)
    if valid_dataloader is not None:
        if hvd.rank() == 0:
            logger.info('Runnning validation on valid data:')
        trainer.validate(valid_dataloader, write_tb=False)
else:
    # run validation, do not write to tensorboard
    if hvd.rank() == 0:
        logger.info('Running validation on train set:')
    trainer.validate(train_dataloader, split='train', write_tb=True)
    if valid_dataloader is not None:
        if hvd.rank() == 0:
            logger.info('Running validation on valid data:')
        trainer.validate(valid_dataloader, write_tb=True)
    # if test_dataloader is not None:
    #     if hvd.rank() == 0:
    #         logger.info('Runnning validation on test data:')
    #     trainer.validate(test_dataloader, write_tb=True)


NameError: name 'train_sampler' is not defined

In [ ]:
xx

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

### Finetune

In [ ]:
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import sys
sys.path.append('..')

### load dataset 

In [ ]:
class Holder:
    def __init__(self):
        pass

In [ ]:
input_seq_len = 512
target_seq_len = 512

num_mem_tokens = 2
input_size = 128

batch_size = 2

args = Holder
args.target_seq_len = target_seq_len
args.input_seq_len = input_seq_len
args.num_mem_tokens = num_mem_tokens
args.input_size = input_size
args.input_prefix = ''
args.block_size = None
args.task_name = 'wikitext-2-v1'

device = 'cpu'

In [ ]:
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [ ]:
from itertools import chain

raw_datasets = datasets.load_dataset('wikitext', args.task_name)
column_names = raw_datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]

def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names,
    desc="Running tokenizer on dataset",
)

block_size = args.input_size 
if args.num_mem_tokens is not None:
    block_size -= 2 * args.num_mem_tokens
history_size = args.input_seq_len - block_size

def group_texts(examples, block_size, history_size=None):
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])

    if history_size is None:
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
    else:
        result = {
            k: [t[max({0, i - history_size}) : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
    result["labels"] = result["input_ids"].copy()
    return result

Found cached dataset wikitext (/home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-389b922bfc5fe729.arrow
Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-6067a66e735cfbb1.arrow
Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-941845a5470f2db7.arrow


In [ ]:
id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

In [ ]:
block_size, history_size

(124, 388)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# def collate_fn(batch):
#     input_ids = [torch.tensor(b['input_ids'][::-1]) for b in batch]
#     labels = [torch.tensor(b['labels'][::-1]) for b in batch]
#     attention_mask = [torch.tensor(b['attention_mask'][::-1]) for b in batch]
#     input_ids = pad_sequence(input_ids, padding_value=id_pad_value).T.flip(1)
#     labels = pad_sequence(labels, padding_value=-100).T.flip(1)
#     attention_mask = pad_sequence(attention_mask, padding_value=0).T.flip(1)

#     collated = {'input_ids': input_ids,
#                 'labels': labels, 
#                 'attention_mask': attention_mask}

#     if input_ids.shape[1] != block_size:
#         labels_mask = torch.ones_like(input_ids, dtype=bool)
#         labels_mask[:, :-block_size] = False
#         collated['labels_mask'] = labels_mask

#     return collated


def collate_fn(batch):
    input_ids = [torch.tensor(b['input_ids']) for b in batch]
    labels = [torch.tensor(b['labels']) for b in batch]
    labels_mask = [torch.ones_like(l, dtype=bool) for l in labels]
    attention_mask = [torch.tensor(b['attention_mask']) for b in batch]

    input_ids = pad_sequence(input_ids, padding_value=id_pad_value).T
    labels = pad_sequence(labels, padding_value=-100).T
    labels_mask = pad_sequence(labels_mask, padding_value=False).T
    attention_mask = pad_sequence(attention_mask, padding_value=0).T

    collated = {'input_ids': input_ids,
                'labels': labels, 
                'labels_mask': labels_mask,
                'attention_mask': attention_mask}

    # if args.vary_n_segments:
    #     n_segments = np.random.randint(1, args.max_n_segments + 1)
    #     n_tokens = n_segments * block_size
    #     for k in collated:
    #         collated[k] = collated[k][:, -n_tokens:]

    return collated


train_dataset = tokenized_datasets["train"].map(lambda x: group_texts(x, block_size, history_size), 
                                        batched=True, desc=f"Grouping train in chunks of {block_size} and history {history_size}")
valid_dataset = tokenized_datasets["validation"].map(lambda x: group_texts(x, block_size), 
                                        batched=True, desc=f"Grouping valid in chunks of {block_size}")


# shuffle train data each epoch (one loop over train_dataset)
# train_sampler = DistributedStrain_dataset[i] for i in range(4)ampler(train_dataset, rank=hvd.rank(), num_replicas=hvd.size(), shuffle=True,
#                                     drop_last=False, seed=args.seed)
# per_worker_batch_size = args.batch_size * args.gradient_accumulation_steps
# global_batch_size = per_worker_batch_size * hvd.size()

# train_sampler = RandomSampler(train_dataset)
kwargs = {'pin_memory': True}#, 'num_workers': args.data_n_workers}
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, **kwargs)

Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-1504f9373e317eca.arrow
Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-c6da793e710ea6d8.arrow


In [ ]:
b = [train_dataset[i] for i in range(4)]

In [ ]:
for k in b[0]:
    b[0][k] = b[0][k][:124]

In [ ]:
batch = collate_fn(b)

In [ ]:
gen = iter(train_dataloader)
batch = next(gen)
batch = next(gen)
batch = next(gen)
batch['input_ids'].shape

torch.Size([2, 512])

In [ ]:
# raw_datasets['train'][1]

### Model

In [ ]:
import math
import torch
from torch.nn import CrossEntropyLoss

In [ ]:
from transformers.modeling_outputs import CausalLMOutputWithCrossAttentions

class MemoryCell(torch.nn.Module):
    def __init__(self, base_model, num_mem_tokens):
        super().__init__()
        self.model = base_model
        self.create_memory(num_mem_tokens)

    def create_memory(self, num_mem_tokens):
        self.num_mem_tokens = num_mem_tokens
        embeddings = self.model.get_input_embeddings()
        memory_weights = torch.randn((num_mem_tokens, self.model.config.n_embd)) * embeddings.weight.data.std()
        self.register_parameter('memory', torch.nn.Parameter(memory_weights, requires_grad=True))

        self.read_memory_position = range(num_mem_tokens)
        self.write_memory_position = range(-num_mem_tokens, 0)

    def set_memory(self, input_shape):
        memory = self.memory.repeat(input_shape[0], 1, 1)
        return memory

    def forward(self, input_ids, memory_state=None, **kwargs):
        if memory_state is None:
            memory_state = self.set_memory(input_ids.shape)

        seg_kwargs = self.process_input(input_ids, memory_state, **kwargs)
        out = self.model(**seg_kwargs)
        out, new_memory_state = self.process_output(out, **kwargs)

        return out, new_memory_state
    
    def process_input(self, input_ids, memory_state, **kwargs):
        seg_kwargs = dict(**kwargs)

        inputs_embeds = kwargs.get('inputs_embeds')
        if inputs_embeds is None:
            inputs_embeds = self.model.get_input_embeddings()(input_ids)
        inputs_embeds = torch.cat([memory_state, inputs_embeds, memory_state], dim=1)

        seg_kwargs['input_ids'] = None
        seg_kwargs['inputs_embeds'] = inputs_embeds
        if kwargs.get('attention_mask') is not None:
            seg_kwargs['attention_mask'] = self.pad_attention_mask(kwargs['attention_mask'], inputs_embeds.shape)
        seg_kwargs['output_hidden_states'] = True
        return seg_kwargs
    
    def pad_attention_mask(self, attention_mask, shape):
        if self.num_mem_tokens in {0, None}:
            return attention_mask
        else:
            mask = torch.ones(*shape[:2], dtype=torch.int64).to(attention_mask.device)
            mask[:, self.num_mem_tokens:-self.num_mem_tokens] = attention_mask
            return mask
    
    def process_output(self, model_outputs, **kwargs):
        if self.num_mem_tokens not in {0, None}:
            out = CausalLMOutputWithCrossAttentions()
            memory_state = model_outputs.hidden_states[-1][:, -self.num_mem_tokens:]
            out['logits'] = model_outputs.logits[:, self.num_mem_tokens:-self.num_mem_tokens]
            
            if kwargs.get('output_hidden_states'):
                out['hidden_states'] = [lh[:, self.num_mem_tokens:-self.num_mem_tokens] for lh in model_outputs.hidden_states]
            if kwargs.get('output_attentions'):
                out['attentions'] = model_outputs['attentions']
        else:
            memory_state = None
            out = model_outputs
            
        return out, memory_state 


In [ ]:
class RecurrentWrapper(torch.nn.Module):
    def __init__(self, memory_cell, **rmt_kwargs):
        super().__init__()
        self.memory_cell = memory_cell
        self.rmt_config = rmt_kwargs

    def forward(self, input_ids, labels=None, labels_mask=None, inputs_embeds=None, attention_mask=None, output_attentions=None, output_hidden_states=None):
        memory_state = None
        segmented = self.segment(input_ids=input_ids, inputs_embeds=inputs_embeds, attention_mask=attention_mask)

        cell_outputs = []
        for seg_num, segment in enumerate(segmented):
            cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)
            cell_outputs.append(cell_out)
            self.manage_gradients(memory_state, seg_num)

        out = self.process_outputs(cell_outputs, labels=labels, 
                                   labels_mask=labels_mask,
                                   output_attentions=output_attentions, 
                                   output_hidden_states=output_hidden_states)
        return out

    def segment(self, **kwargs):
        segments = []
        for k, tensor in kwargs.items():
            if tensor is not None:
                k_segments = self.split_tensor(tensor)
                for s, k_seg in enumerate(k_segments):
                    if s < len(segments):
                        segments[s][k] = k_seg
                    else:
                        segments.append({k: k_seg})

        return segments
    
    def split_tensor(self, tensor):
        align = self.rmt_config.get('segment_alignment')
        segment_size = self.rmt_config.get('segment_size')
        if align in {'left', None}:
            split_inds = list(range(0, tensor.shape[1], segment_size)) + [tensor.shape[1]]
            segments = [tensor[:, start:end] for (start, end) in zip(split_inds, split_inds[1:])]
        elif align in {'right', None}:
            split_inds = (list(range(tensor.shape[1], 0, -segment_size)) + [0])[::-1]
            segments = [tensor[:, start:end] for (start, end) in zip(split_inds, split_inds[1:])]
        elif align == 'center':
            n_seg = math.ceil(tensor.shape[1] / segment_size)
            segments = torch.chunk(tensor, n_seg, dim=1)
        else:
            raise NotImplementedError
        return segments

    def process_outputs(self, cell_outputs, **kwargs):
        out = CausalLMOutputWithCrossAttentions()
        full_logits = torch.cat([o.logits for o in cell_outputs], dim=1)
        full_hidden_states = tuple([torch.cat(layer_hs, dim=1) for layer_hs in zip(*[o.hidden_states for o in cell_outputs])])

        labels = kwargs.get('labels')
        if labels is not None:
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = full_logits[..., :-1, :].contiguous()
            flat_labels = shift_labels.view(-1)
            flat_logits = shift_logits.view(-1, shift_logits.size(-1))
            
            loss_fct = CrossEntropyLoss()
            labels_mask = kwargs.get('labels_mask')
            if labels_mask is not None:
                shift_mask = labels_mask[..., :-1].contiguous()

                flat_labels = flat_labels[shift_mask.view(-1)]
                flat_logits = flat_logits[shift_mask.view(-1)]
                
            out['loss'] = loss_fct(flat_logits, flat_labels)

        out['logits'] = full_logits
        segment_keys = ['loss', 'logits']
        if kwargs.get('output_attentions'):
            segment_keys.append('attentions')
        if kwargs.get('output_hidden_states'):
            segment_keys.append('hidden_states')
            out['hidden_states'] = full_hidden_states

        # for seg_num, o in enumerate(cell_outputs):
        #     for key, value in o.items():
        #         if any([sk in key for sk in segment_keys]):
        #             out[f'{key}_{seg_num}'] = value

        return out 
        
    def manage_gradients(self, memory_state, seg_num):
        k2, max_n_segments = self.rmt_config.get('k2'), self.rmt_config.get('max_n_segments')
        if seg_num == 0 \
            or k2 in {-1, None} \
            or seg_num + k2 > max_n_segments:
                return True
        
        memory_state = memory_state.detach()
        return False

In [ ]:

num_mem_tokens = 10
# device = torch.device(3)
device = 'cpu'

rmt_config = {'num_mem_tokens': 10, 
               #  'max_n_segments': 1,
               #  'segment_alignment': 'right',
               #  'tokenizer': tokenizer,
               #  'memory_layers': 'all', 
               #  'share_memory_layers': True,
               #  'reconstruction_loss_coef': 0.1,
               #  'k1': -1, 'k2': 3,
               #  'segment_ordering': 'regular',
               #  'input_size': 1024, 
               #  'bptt_depth': -1, 
               #  'sum_loss': False,
             }

base_model = AutoModelForCausalLM.from_pretrained(model_name)
cell = MemoryCell(base_model, num_mem_tokens=2)

/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [ ]:
rmt = RecurrentWrapper(cell, max_n_segments=5, segment_size=124, segment_alignment='center')

In [ ]:
gen = iter(train_dataloader)
batch = next(gen)
batch = next(gen)
batch = next(gen)
batch.pop('labels_mask')
# batch.pop('labels')
1

1

In [ ]:
rmt_out = rmt(**batch)

In [ ]:
rmt_out.keys()

odict_keys(['loss', 'logits'])

# Contract-NLI

In [ ]:
from datasets import Dataset

class CNLIDataset(Dataset):
    def __init__(self, json_path):
        with open(json_path, 'r') as f:
            self.samples = json.load(f)

    def __getitem__(self, item):
        return self.samples[item]
    
    def __len__(self):
        return len(self.samples)

In [ ]:
dataset = CNLIDataset('/cephfs/home/bulatov/bulatov/datasets/contract_nli/contract-nli/test_processed.json')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0032s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0160s.) Setting batch_size=4.
[Parallel(n_jobs=12)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  50 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 123 out of 123 | elapsed:    0.1s finished


In [ ]:
args.input_seq_len

512

In [ ]:
from torch.nn.utils.rnn import pad_sequence

tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_tokens('[GEN]', special_tokens=True)
gen_token = tokenizer.encode('[GEN]')[0]
id_pad_value = tokenizer.eos_token_id

def collate_fn(batch):
    inputs = [b['context'][:args.input_seq_len * 10] for b in batch]
    labels = [b['answer'][:args.input_seq_len * 10] for b in batch]

    collated = {}
    inputs = tokenizer.batch_encode_plus(list(inputs), padding=False)
    labels = tokenizer.batch_encode_plus(list(labels), padding=False)

    full_inputs, labels_mask = [], []
    for inp, lab in zip(inputs['input_ids'], labels['input_ids']):
        inp = inp[:args.input_seq_len - len(lab) - 1]
        full_inputs.append(torch.tensor(inp + [gen_token] + lab))
        labels_mask.append(torch.tensor([False] * len(inp) + [True] * (len(lab) + 1)))

    full_inputs = pad_sequence(full_inputs, padding_value=id_pad_value).T
    labels_mask = pad_sequence(labels_mask, padding_value=False).T

    collated['input_ids'] = collated['labels'] = full_inputs
    collated['labels_mask'] = labels_mask
    collated['attention_mask'] = collated['input_ids'] != id_pad_value
    return collated

In [ ]:
batch = collate_fn([dataset[i] for i in range(4)])
# batch = [dataset[i] for i in range(4)]

In [ ]:
segmented = rmt.segment(**batch)

In [ ]:
[s['input_ids'].shape for s in segmented]

[torch.Size([4, 103]),
 torch.Size([4, 103]),
 torch.Size([4, 103]),
 torch.Size([4, 103]),
 torch.Size([4, 100])]

In [ ]:
labels = batch.pop('labels')
rmt_out = rmt(**batch)

In [ ]:
rmt_out.logits.shape

torch.Size([4, 512, 50258])

In [ ]:
rmt_out.logits[batch['labels_mask']].shape

torch.Size([20, 50258])

In [ ]:
labels[batch['labels_mask']].shape

torch.Size([20])

In [ ]:
tokenizer.decode([ 3673,    44,  1463,   276, 50256])

'NotMentioned<|endoftext|>'

In [ ]:
tokenizer.decode([14539,   603,   434])

'Entailment'

In [ ]:
# rmt_out = rmt(**batch)

self = rmt
input_ids = batch['input_ids']

memory_state = None
segmented = self.segment(input_ids=input_ids, inputs_embeds=None, attention_mask=None)

cell_outputs = []
for seg_num, segment in enumerate(segmented):
    cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)#**batch)
    cell_outputs.append(cell_out)
    self.manage_gradients(memory_state, seg_num)

# out = self.process_outputs(cell_outputs)

In [ ]:
input_ids.shape

torch.Size([2, 248])

In [ ]:
torch.chunk(torch.ones(10), 3)

(tensor([1., 1., 1., 1.]), tensor([1., 1., 1., 1.]), tensor([1., 1.]))

### seq2seq

In [ ]:
from huggingface_hub import hf_hub_download
# scrolls_metric_path = hf_hub_download(repo_id="datasets/tau/scrolls", filename="metrics/scrolls.py")tokenizer.add_tokens('[GEN]', special_tokens=True)
gen_token = tokenizer.encode('[GEN]')[0]

rmt.memory_cell.model.resize_token_embeddings(len(tokenizer))

# gen_token = tokenizer.eos_token_id

Embedding(50258, 768)

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [ ]:
global_attention_first_token = False  # should be True for LED
encode_plus_kwargs = {'truncation': True, 'padding': 'longest', 'pad_to_multiple_of': 1}
# generate_kwargs = {'max_length': args.targettokenizer.add_tokens('[GEN]', special_tokens=True)
# gen_token = tokenizer.encode('[GEN]')[0]

# rmt.memory_cell.model.resize_token_embeddings(len(tokenizer))_seq_len, 'min_length': args.target_seq_len}
generate_kwargs = {}

from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    inputs = [b['input'][:args.input_seq_len * 10] for b in batch]
    labels = [b['output'][:args.input_seq_len * 10] for b in batch]

    collated = {}
    inputs = tokenizer.batch_encode_plus(list(inputs), padding=False)
    labels = tokenizer.batch_encode_plus(list(labels), padding=False)

    full_inputs = [torch.tensor(i[:input_size - len(l) - 1] + [gen_token] + l) for i, l in zip(inputs['input_ids'], labels['input_ids'])]
    full_inputs = pad_sequence(full_inputs, padding_value=tokenizer.pad_token_id).T
    
    labels_mask = torch.zeros_like(full_inputs).bool()
    for i, l in enumerate(labels['input_ids']):
        labels_mask[i, -len(l) -1:] = True
    collated['input_ids'] = collated['labels'] = full_inputs
    collated['labels_mask'] = labels_mask
    collated['attention_mask'] = collated['input_ids'] != tokenizer.pad_token_id

    return collated
    

In [ ]:
seq2seq_task_name = 'quality'
dataset = datasets.load_dataset('tau/scrolls', seq2seq_task_name)
train_dataset = dataset['train']
# shuffle train data each epoch (one loop over train_dataset)
# train_sampler = DistributedSampler(train_dataset, shuffle=True, drop_last=False, seed=args.seed)
kwargs = {'pin_memory': True}
train_dataloader = DataLoader(train_dataset, batch_size=2,
                                collate_fn=collate_fn, **kwargs)

Found cached dataset scrolls (/home/bulatov/.cache/huggingface/datasets/tau___scrolls/quality/1.0.0/672021d5d8e1edff998a6ea7a5bff35fdfd0ae243e7cf6a8c88a57a04afb46ac)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
gen = iter(train_dataloader)
batch = next(gen)

In [ ]:
batch.keys()

dict_keys(['input_ids', 'labels', 'labels_mask', 'attention_mask'])

In [ ]:
batch['input_ids'].shape

torch.Size([2, 128])

In [ ]:
batch['labels'].shape

torch.Size([2, 128])

In [ ]:
tokenizer.batch_decode(batch['input_ids'])

['Why is Si retirement so significant to the Space Exploration Team? \n\n (A) There aren’t enough working people in the world. They won’t be able to find a replacement.\n (B) As one of two remaining spacemen, it would likely mean the defunding and shut down of the Space Exploration Team.\n (C) Training new spacemen is costly and time consuming. They won’t have anyone else ready after him.\n (D) His retirement may inspire others to[GEN]Training new spacemen is costly and time consuming. They won’t have anyone else ready after him.',
 'What makes Gubelin an outlier in the present day?\n\n (A) He is much older than the rest of the population.\n (B) He refuses new operations that could improve his health.\n (C) His mind is still active, and he values hard work.\n (D) He still wears glasses and value objects like the gold watch given to Si.\n\n\nSPACEMAN ON A SPREE\n\n\n\n\n   BY MACK REYNOLDS\n\n\n\n\n   Illustrated by Nodel\n\n\n\n\n[GEN]He still wears glasses and value objects like the g

In [ ]:
tokenizer.batch_decode([c[i] for c, i in zip(batch['input_ids'], batch['labels_mask'])])

['[GEN]Training new spacemen is costly and time consuming. They won’t have anyone else ready after him.',
 '[GEN]He still wears glasses and value objects like the gold watch given to Si.']

In [ ]:
tokenizer.batch_decode([c[i] for c, i in zip(batch['input_ids'], batch['attention_mask'])])

['Why is Si retirement so significant to the Space Exploration Team? \n\n (A) There aren’t enough working people in the world. They won’t be able to find a replacement.\n (B) As one of two remaining spacemen, it would likely mean the defunding and shut down of the Space Exploration Team.\n (C) Training new spacemen is costly and time consuming. They won’t have anyone else ready after him.\n (D) His retirement may inspire others toTraining new spacemen is costly and time consuming. They won’t have anyone else ready after him.',
 'What makes Gubelin an outlier in the present day?\n\n (A) He is much older than the rest of the population.\n (B) He refuses new operations that could improve his health.\n (C) His mind is still active, and he values hard work.\n (D) He still wears glasses and value objects like the gold watch given to Si.\n\n\nSPACEMAN ON A SPREE\n\n\n\n\n   BY MACK REYNOLDS\n\n\n\n\n   Illustrated by Nodel\n\n\n\n\nHe still wears glasses and value objects like the gold watch 

In [ ]:
out = rmt(**batch)

In [ ]:
out.loss

tensor(60.2067, grad_fn=<NllLossBackward>)

In [ ]:
out.loss

tensor(61.4377, grad_fn=<NllLossBackward>)